In [32]:
import os

import analysis_session_helper_functions_v2 as helper
import analysis_utils as utils

import pandas as pd

In [33]:
cohort = 'cohort_2'
to_analyze = '510_clean'
data_dir = '/Users/rebekahzhang/Documents/shuler_lab/behavior_data'
data_folder = os.path.join(data_dir, cohort, to_analyze)
print(data_folder)

/Users/rebekahzhang/Documents/shuler_lab/behavior_data/cohort_2/510_clean


# Generates all session logs

does not need to be run if session logs have been generated

In [34]:
session_log = helper.generate_all_session_log(data_folder)
session_log.head()

,date,mouse,dir,filename
0,2023-05-12,RZ012,2023-05-12_10-29-00_RZ012,data_RZ012_2023-05-12_10-29-00.txt
1,2023-05-10,RZ014,2023-05-10_12-03-39_RZ014,data_RZ014_2023-05-10_12-03-39.txt
2,2023-05-16,RZ016,2023-05-16_12-05-31_RZ016,data_RZ016_2023-05-16_12-05-31.txt
3,2023-05-18,RZ016,2023-05-18_14-13-53_RZ016,data_RZ016_2023-05-18_14-13-53.txt
4,2023-05-13,RZ016,2023-05-13_14-26-33_RZ016,data_RZ016_2023-05-13_14-26-33.txt


get a list of mouse names

In [35]:
# prints mouse names in data folder, check for weird ones and delete from data base
mouse_list = session_log.mouse.unique().tolist()
mouse_list.sort()
print(mouse_list)

['RZ012', 'RZ013', 'RZ014', 'RZ015', 'RZ016', 'RZ017']


get the type of training for session meta data and add to log

In [36]:
training_list = []
for dir_name, file_name in zip(session_log.dir, session_log.filename):
    session_meta = utils.load_session_meta(data_folder, dir_name, file_name)
    training = session_meta.training.tolist()[0]
    training_list.append(training)

session_log['training'] = training_list

add columns of basic info to each session to log

In [37]:
session = utils.load_session(data_folder, dir_name, file_name)

In [38]:
session_basics_columns = ['num_blocks', 'num_trials', 'rewards', 'time', 'proper_end']
for dir_name, file_name in zip(session_log.dir, session_log.filename):
    session = utils.load_session(data_folder, dir_name, file_name)
    session_basic = helper.get_session_basics(session)
    session_log.loc[session_log.dir == dir_name, session_basics_columns] = session_basic

### Focus only on regular training sessions

In [39]:
training_session_log = session_log.loc[session_log.training == 'regular'].sort_values('dir').reset_index()
training_session_log.head()

,index,date,mouse,dir,filename,training,num_blocks,num_trials,rewards,time,proper_end
0,7,2023-05-10,RZ012,2023-05-10_10-32-00_RZ012,data_RZ012_2023-05-10_10-32-00.txt,regular,3.0,210.0,695.09,1852.88,False
1,50,2023-05-10,RZ013,2023-05-10_11-08-16_RZ013,data_RZ013_2023-05-10_11-08-16.txt,regular,5.0,299.0,653.90,1967.93,True
2,1,2023-05-10,RZ014,2023-05-10_12-03-39_RZ014,data_RZ014_2023-05-10_12-03-39.txt,regular,5.0,289.0,719.52,1994.20,True
3,8,2023-05-10,RZ015,2023-05-10_13-11-46_RZ015,data_RZ015_2023-05-10_13-11-46.txt,regular,3.0,188.0,499.15,1417.92,True
4,11,2023-05-10,RZ016,2023-05-10_13-37-41_RZ016,data_RZ016_2023-05-10_13-37-41.txt,regular,5.0,272.0,646.67,1864.71,True


### Examine quality of sessions
doesn't need to run when data folder is cleaned

check for missing sessions by the number of sessions in each training day

In [40]:
num_mice = 6
date_list = training_session_log.date.unique().tolist()
for date in date_list:
    data = training_session_log.loc[training_session_log['date'] == date]
    if len(data) < num_mice:
        print(date)

2023-05-13
2023-05-16
2023-05-17
2023-05-18
2023-05-23
2023-05-24
2023-05-25
2023-05-26


check for same mouse with multiple sessions per day, prints out date and mouse if too many

In [41]:
# prints dates of the same mouse with multiple sessions
for d in training_session_log.date.unique().tolist():
    session_of_the_day = training_session_log.loc[training_session_log['date'] == d]
    for mouse in mouse_list:
        count = len(session_of_the_day.loc[session_of_the_day['mouse'] == mouse])
        if count > 1:
            print(d, mouse)   

check for short sessions, prints out dir

In [42]:
short_session = training_session_log.loc[(training_session_log['training'] == 'regular') & 
                                         (training_session_log['num_trials'] < 100)] 
print(short_session.dir)

Series([], Name: dir, dtype: object)


### Add number of days in training to training log

In [43]:
for mouse in mouse_list:
    total_days = sum(training_session_log.mouse == mouse)
    training_session_log.loc[training_session_log.mouse == mouse, 'days'] = list(range(total_days))

In [44]:
training_session_log.days.max()

10.0

### Saves all sessions log and training session log

In [45]:
filename = f'all_sessions.csv'
path = os.path.join(data_folder, filename)
session_log.to_csv(path)

In [46]:
filename = f'training_sessions.csv'
path = os.path.join(data_folder, filename)
training_session_log.to_csv(path)

# Generate all trials per session

load session log

In [47]:
training_session_log = utils.load_session_log(data_folder, 'training_sessions.csv')

### Raw data processing and generate initial all trials df
adds trial numbers and states to hardware entries
create all trials df with trial basics added
both files saved, 
does't need to be rerun

In [48]:
all_trials_column_names = ['session_trial_num', 'block_trial_num', 'block_num', 'start_time', 'end_time']

In [49]:
# for dir_name, file_name in zip(training_session_log.dir, training_session_log.filename):
    
#     processed_path = os.path.join(data_folder, dir_name, f'processed_{file_name[:-4]}.csv')
#     all_trials_path =os.path.join(data_folder, dir_name, f'{dir_name}_all_trials.csv')
#     # if os.path.isfile(processed_path) and os.path.isfile(all_trials_path):
#     #     continue

#     session = utils.load_session(data_folder, dir_name, file_name)
#     total_trial_list = helper.generate_total_trial_list(training_session_log, dir_name)
#     all_trials = pd.DataFrame(columns=all_trials_column_names)

#     for t in total_trial_list:
#         trial = session.loc[session['session_trial_num'] == t]
        
#         trial_basics = helper.get_trial_basics(trial)
#         helper.align_trial_number(session, trial_basics)
        
#         trial_state_times = helper.get_trial_state_times(trial)
#         helper.align_trial_states(session, trial_state_times, trial_basics)

#         trial_basics = pd.DataFrame([trial_basics])
#         all_trials = pd.concat([all_trials, trial_basics], ignore_index=True)
    
#     session = utils.trim_session(training_session_log, dir_name, session)
#     session.to_csv(processed_path)
#     all_trials.to_csv(all_trials_path)

In [50]:
for dir_name, file_name in zip(training_session_log.dir, training_session_log.filename):
    
    processed_path = os.path.join(data_folder, dir_name, f'processed_{file_name[:-4]}.csv')
    all_trials_path =os.path.join(data_folder, dir_name, f'{dir_name}_all_trials.csv')
    # if os.path.isfile(processed_path) and os.path.isfile(all_trials_path):
    #     continue

    session = utils.load_session(data_folder, dir_name, file_name)
    session['trial_time'] = ''
    total_trial_list = helper.generate_total_trial_list(training_session_log, dir_name)
    all_trials = pd.DataFrame(columns=all_trials_column_names)

    for t in total_trial_list:
        trial = session.loc[session['session_trial_num'] == t]
        
        trial_basics = helper.get_trial_basics(trial)
        helper.align_trial_number(session, trial_basics)
        
        trial_state_times = helper.get_trial_state_times(trial)
        helper.align_trial_states(session, trial_state_times, trial_basics)

        trial = session.loc[session['session_trial_num'] == t]
        helper.add_trial_time(session, t, trial, trial_basics)

        trial_basics = pd.DataFrame([trial_basics])
        all_trials = pd.concat([all_trials, trial_basics], ignore_index=True)
    
    session = utils.trim_session(training_session_log, dir_name, session)
    session.to_csv(processed_path)
    all_trials.to_csv(all_trials_path)

### Adding analyzed trial data to all trials df

In [51]:
all_trials_data_column = ['bg_drawn', 'blk_type', 'bg_length', 'enl_repeats', 
                          'miss_trial', 'time_waited', 'reward', 'num_consumption_lick']

In [52]:
for dir_name, file_name in zip(training_session_log.dir, training_session_log.filename):
    all_trials_analyzed_path =os.path.join(data_folder, dir_name, f'{dir_name}_all_trials_analyzed.csv')
    # if os.path.isfile(all_trials_analyzed_path):
    #     continue

    processed_session = utils.load_processed_session(data_folder, dir_name, file_name)
    all_trials = utils.load_all_trials(data_folder, dir_name)
    total_trial_list = helper.generate_total_trial_list(training_session_log, dir_name)

    for t in total_trial_list:
        trial = processed_session.loc[processed_session['session_trial_num'] == t]
        trial_performance = helper.get_trial_performance(trial)
        all_trials.loc[all_trials['session_trial_num'] == t, all_trials_data_column] = trial_performance
    
    all_trials.to_csv(all_trials_analyzed_path)

# Generate all blocks per session

load session log

In [53]:
training_session_log = utils.load_session_log(data_folder, 'training_sessions.csv')

### generate all blocks df and saves to raw data folder

In [54]:
all_blocks_column_names = ['block_num', 'blk_type', 'num_trials', 'start_time', 'end_time',
                           'bg_drawn_mean', 'bg_drawn_std', 'bg_length_mean', 'bg_length_std', 
                           'enl_repeats_mean', 'enl_repeats_std', 'num_miss_trials', 'time_waited_mean', 
                           'time_waited_std', 'reward_mean', 'reward_std', 'num_consumption_lick_mean', 
                           'num_consumption_lick_std']

In [55]:
def generate_all_blocks_df(column_names, total_block_list):
    """
    makes an empty df with each row being a trial, and each column with trial info
    trial number is added to the df
    """
    all_blocks = pd.DataFrame(columns=column_names)
    all_blocks['block_num'] = total_block_list
    return all_blocks

In [56]:
for dir_name, file_name in zip(training_session_log.dir, training_session_log.filename):
    filename = f'{dir_name}_all_blocks.csv'
    path = os.path.join(data_folder, dir_name, filename)
    # if os.path.isfile(path):
    #     continue

    all_trials_analyzed = utils.load_all_trials_analyzed(data_folder, dir_name)
    total_block_list = helper.generate_total_block_list(training_session_log, dir_name)
    all_blocks = helper.generate_all_blocks_df(all_blocks_column_names, total_block_list)
    
    for blk in total_block_list:
        block = all_trials_analyzed.loc[all_trials_analyzed['block_num'] == blk]
        block_data = helper.get_block_data(block)
        all_blocks.loc[all_blocks.block_num == blk, all_blocks_column_names[1:]] = block_data
        
    all_blocks.to_csv(path)

In [57]:
all_blocks

,block_num,blk_type,num_trials,start_time,end_time,bg_drawn_mean,bg_drawn_std,bg_length_mean,bg_length_std,enl_repeats_mean,enl_repeats_std,num_miss_trials,time_waited_mean,time_waited_std,reward_mean,reward_std,num_consumption_lick_mean,num_consumption_lick_std
0,0,l,45.0,1685114421.068404,1685114752.382228,3.034783,0.380186,3.429381,0.683776,3.065217,2.416629,0,0.773045,0.542357,1.381196,0.884345,8.217391,2.169619
1,1,s,42.0,1685114752.382339,1685114969.523603,1.016279,0.127111,1.174444,0.355013,1.906977,1.98583,0,0.875303,0.467188,1.567524,0.744982,9.465116,1.931553
2,2,l,44.0,1685114969.523719,1685115289.198917,3.008889,0.333591,3.377476,0.626661,3.266667,2.87149,0,0.726367,0.545128,1.302069,0.833534,8.977778,2.378778
3,3,s,43.0,1685115289.19905,1685115505.474034,1.025,0.118371,1.220792,0.417409,2.090909,1.926778,0,0.694501,0.584182,1.237932,0.947504,10.113636,2.479834
4,4,l,59.0,1685115505.474158,1685115923.904352,2.975,0.353973,3.343793,0.569845,3.166667,2.443717,0,0.629993,0.447494,1.148288,0.747694,9.666667,2.311846
5,5,s,50.0,1685115923.904473,1685116174.840127,0.986275,0.12332,1.350955,0.518832,3.078431,2.674645,0,0.569298,0.404382,1.046797,0.694711,10.392157,2.836043
6,6,l,52.0,1685116174.840242,1685116547.932732,3.016981,0.370413,3.38581,0.635427,3.320755,2.636822,0,0.653626,0.608406,1.162584,0.948872,9.679245,3.256789
7,7,s,42.0,1685116547.932849,1685116753.708108,1.016279,0.123308,1.264468,0.47014,2.534884,2.271446,0,0.520948,0.327584,0.970126,0.564411,9.255814,2.498505


In [58]:
test = utils.load_all_blocks(data_folder, training_session_log.dir[2])

In [59]:
test

,Unnamed: 0,block_num,blk_type,num_trials,start_time,end_time,bg_drawn_mean,bg_drawn_std,bg_length_mean,bg_length_std,enl_repeats_mean,enl_repeats_std,num_miss_trials,time_waited_mean,time_waited_std,reward_mean,reward_std,num_consumption_lick_mean,num_consumption_lick_std
0,0,0,l,42.0,1.683735e+09,1.683735e+09,3.000000,0.345722,3.248847,0.582061,2.372093,2.319927,2,1.863680,1.400934,2.830320,1.638421,10.146341,4.350638
1,1,1,s,59.0,1.683735e+09,1.683735e+09,0.976667,0.130665,1.339843,0.521957,3.116667,3.042588,0,1.211890,0.797704,2.045598,1.195985,11.416667,4.026571
2,2,2,l,40.0,1.683735e+09,1.683736e+09,2.956098,0.310684,3.263254,0.519092,3.000000,2.439262,0,1.190850,0.937938,1.980812,1.294088,10.707317,4.081935
3,3,3,s,39.0,1.683736e+09,1.683736e+09,0.992500,0.128876,1.264156,0.522154,2.700000,3.314305,0,1.353147,0.898095,2.240583,1.264877,12.875000,4.772719
4,4,4,l,56.0,1.683736e+09,1.683736e+09,3.008772,0.342408,3.302050,0.654142,2.912281,2.911067,0,1.993592,1.583637,2.938435,1.746578,12.035088,4.594115
5,5,5,s,48.0,1.683736e+09,1.683737e+09,1.022449,0.121218,1.164654,0.338135,1.836735,1.829696,0,1.771342,1.077158,2.799735,1.412002,12.632653,5.036094
